In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install jsonlines
!pip install transformers
!pip install fairseq
!pip install fastBPE

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 47.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 16.3 MB/s eta 0:00:00
  Preparing metadata

In [3]:
import argparse
import torch
import jsonlines
import random
import os
import json
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, get_cosine_schedule_with_warmup
from tqdm import tqdm
from typing import List
from sklearn.metrics import f1_score, precision_score, recall_score

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import time
import re
import random
import seaborn as sns
from tqdm.notebook import tqdm
import torch
import argparse
import pickle
import numpy as np
from os.path import join

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.backends import cudnn
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.manifold import TSNE

from fairseq.models.roberta import RobertaModel
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from transformers.modeling_utils import *
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
from transformers import AutoModel, AutoTokenizer

import argparse

In [5]:
warmup_sets = '/content/drive/MyDrive/pRoBERTa/data/converted_data_warmup.json'
train_sets = '/content/drive/MyDrive/pRoBERTa/data/converted_data_train.json'
dest = '/content/drive/MyDrive/pRoBERTa/trained_module/labels_prediction/2class'
model = 'base'
epochs = 20
batch_size_gpu = 8
batch_size_accumulated = 256
lr_base = 1e-5
lr_linear = 1e-3

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device "{device}"')

Using device "cuda"


# Load data

In [9]:
class LoadDataFromJson(Dataset):
    def preprocess_text(self, text: str) -> str:
        text = re.sub(r"['\",\.\?:\-!]", "", text)
        text = text.strip()
        text = " ".join(text.split())
        text = text.lower()
        return text
    def __init__(self, datasets):
        self.samples = []

        with open(datasets) as file:
            data = json.load(file)

        for id, item in data.items():
            sentences = item["context"]
            if item['verdict'] != 'NEI':
                data_sample = {}

                data_sample['id'] = id
                data_sample['claim'] = item['claim']
                data_sample['verdict'] = item['verdict']
                data_sample['evidence'] = self.preprocess_text(item['evidence'])
                data_sample['domain'] = item['domain']

                self.samples.append(data_sample)


    def to_dataframe(self):
        return pd.DataFrame(self.samples)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [10]:
warmupsets = LoadDataFromJson(warmup_sets)
df = warmupsets.to_dataframe()
df

,id,claim,verdict,evidence,domain
0,36112,trong năm nay hai địa phương dẫn đầu và gần nh...,SUPPORTED,năm nay hai địa phương có lượng hồ sơ dẫn đầu ...,khoahoc
1,36633,do đó chị đã quyết định mở hàng ốc nhồi hơn mộ...,SUPPORTED,vì vậy sau khi trở lại sài gòn chị đã quyết đị...,dulich
2,19153,bộ tài chính sẽ chịu trách nhiệm hướng dẫn các...,SUPPORTED,các địa phương phải sử dụng hiệu quả dưới sự h...,thoisu
3,12423,vụ việc parra thiêu sống katherine gomez khiến...,SUPPORTED,sự việc khiến người dân peru bị sốc và giận dữ,thegioi
4,19927,dự án xử lý dioxin tại sân bay biên hoà được k...,SUPPORTED,đến nay chính phủ mỹ đã đóng góp 16325 triệu u...,thegioi
...,...,...,...,...,...
527,36158,mô hình tự tổ chức đóng vai trò quan trọng tro...,SUPPORTED,mô hình tự tổ chức góp phần cho phép robot tự ...,khoahoc
528,10965,những đợt triển khai dài ngày trong quân đội m...,SUPPORTED,những đợt triển khai dài ngày đã đẩy tỷ lệ ly ...,thegioi
529,46648,điều khiến du khách ấn tượng khi ghé bãi bùi c...,REFUTED,điều khiến du khách ấn tượng khi ghé bãi bùi c...,dulich
530,13375,chặng launcestonsydney có vấn đề về kĩ thuật v...,SUPPORTED,do vấn đề thời tiết kỹ thuật nên chặng launces...,dulich


In [11]:
trainsets = LoadDataFromJson(train_sets)
df_train = trainsets.to_dataframe()

In [12]:
df_train['verdict'].unique()

array(['SUPPORTED', 'REFUTED'], dtype=object)

In [13]:
df_filter = df_train.loc[df_train['evidence'].apply(lambda x: len(x.split())) < 3 ]

In [14]:
df_filter

,id,claim,verdict,evidence,domain
25,7150,ở khu viên trường cung cấp sinh viên có thể ti...,SUPPORTED,thiên minh,giaoduc
26,7151,đồng thời sinh viên có thể tiết kiệm chi phí s...,SUPPORTED,thiên minh,giaoduc
42,7167,trong số 129210 học sinh tốt nghiệp thcs có 55...,SUPPORTED,thanh hằng,giaoduc
45,7170,hơn 940000 sinh viên quốc tế chọn mỹ là điểm đ...,SUPPORTED,thiên minh,giaoduc
77,7202,học sinh muốn ứng tuyển phải có quốc tịch việt...,SUPPORTED,bình minh,giaoduc
...,...,...,...,...,...
14077,47428,pgsts lê minh hà cho biết theo bước đầu khảo s...,REFUTED,hương thảo,khoahoc
14095,47743,sự kiện lấy ý tưởng về một vở đại nhạc vũ kịch...,REFUTED,thanh thư,dulich
14271,47919,giá vé vào vườn thanh bình 5000 10000 đồng một...,REFUTED,quỳnh mai,dulich
14294,47942,vợ anh kiệt đánh giá ba phần cơm của cả nhà ch...,REFUTED,huỳnh nhi,dulich


In [15]:
df_train

,id,claim,verdict,evidence,domain
0,7125,ngoài việc không giới hạn mức lương công ty cò...,SUPPORTED,công ty cũng có chế độ đãi ngộ tốt có lương th...,giaoduc
1,7126,thành lập năm 2016 phát triển phần mềm tổ chức...,SUPPORTED,faba technology thành lập năm 2016 chuyên cung...,giaoduc
2,7127,nhằm phát triển đội ngũ faba thường tổ chức nh...,SUPPORTED,bên cạnh đó faba thường xuyên tổ chức những dự...,giaoduc
3,7128,sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,SUPPORTED,theo adeco việt nam mức lương cao nhất đối với...,giaoduc
4,7129,theo adeco việt nam mức lương cao nhất đối với...,SUPPORTED,trong khi đó theo báo cáo thị trường nhân lực ...,giaoduc
...,...,...,...,...,...
14379,48027,cô phương cũng đã nhiều lần định bỏ việc làm t...,REFUTED,cô cũng đã nhiều lần định bỏ việc nhất là sau ...,dulich
14380,48028,nhiều kỹ thuật lặn được biến tấu đa dạng khớp ...,REFUTED,nhiều kỹ thuật lặn được biến tấu đa dạng khớp ...,dulich
14381,48029,phần dễ nhất chính là quấy bột dành cho những ...,REFUTED,phần khó nhất chính là quấy bột dành cho những...,dulich
14382,48030,hàng bánh đúc nóng trên phố lê ngọc hân đã có ...,REFUTED,hàng bánh đúc nóng trên phố lê ngọc hân đã có ...,dulich


In [16]:
# Xóa những dòng có evidence < 3 và evidence = 0
df_train = df_train.drop(df_train[df_train.isin(df_filter.to_dict('list')).all(1)].index)

In [31]:
df_train.isna().sum()

id          0
claim       0
verdict     0
evidence    0
domain      0
dtype: int64

In [32]:
df_train.columns

Index(['id', 'claim', 'verdict', 'evidence', 'domain'], dtype='object')

In [33]:
df_train['verdict'].unique()

array(['SUPPORTED', 'REFUTED'], dtype=object)

In [34]:
inputs = df_train.loc[:, ['claim','evidence', 'domain']]
labels = df_train['verdict']

# Chuẩn bị dữ liệu train

In [35]:
# load model phoBert và tokenizer của model đó
model_bert = AutoModel.from_pretrained(f'vinai/phobert-{model}')
tokenizer = AutoTokenizer.from_pretrained(f"vinai/phobert-{model}", use_fast=False)

In [36]:
# Load BPE encoder
parser = argparse.ArgumentParser()
parser.add_argument(
    '--bpe-codes',
    default=f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/bpe.codes',

    required=False,
    type=str,
    help='path to fastBPE BPE'
)

args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file(f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/dict.txt')


In [37]:
# encode label
le = LabelEncoder()
encoded_train_labels = le.fit_transform(labels)
print(le.classes_)

['REFUTED' 'SUPPORTED']


In [38]:
# chia tập train thành 2 phần train và validate với tỉ lệ 9:1
from sklearn.model_selection import train_test_split

train_sents, val_sents, train_labels, val_labels = train_test_split(inputs, encoded_train_labels, test_size=0.1, random_state=1)


In [39]:
# Thực hiện chuyển dataset ban đầu thành subword index
MAX_LEN = 256

train_ids = []
print('Processing train set ...')
for index, row in train_sents.iterrows():
  claim_value = row['claim']
  evidence_value = row['evidence']
  domain_value = row['domain']
  subwords = ' ' + bpe.encode(claim_value) + bpe.encode(evidence_value) + bpe.encode(domain_value) + ' '
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
  train_ids.append(encoded_sent)

val_ids = []
print('Processing test set ...')
for index, row in val_sents.iterrows():
  claim_value = row['claim']
  evidence_value = row['evidence']
  domain_value = row['domain']
  subwords = ' ' + bpe.encode(claim_value) + bpe.encode(evidence_value) + bpe.encode(domain_value) + ' '
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
  val_ids.append(encoded_sent)

Processing train set ...
Processing test set ...


In [40]:
# thêm padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype='long', value=1, truncating='post', padding='post')
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype='long', value=1, truncating='post', padding='post')


In [41]:
# Tạo mask để BERT không chú ý tới padding (pad id = 1)
train_masks = []
for sent in train_ids:
  mask = [int(token_id != 1) for token_id in sent]
  train_masks.append(mask)

val_masks = []
for sent in val_ids:
  mask = [int(token_id != 1) for token_id in sent]
  val_masks.append(mask)

In [42]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 4

# convert sang tensor
train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

# tạo data loader để train
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE)

In [43]:
val_labels.size()

torch.Size([1406])

In [44]:
train_labels.size()

torch.Size([12654])

# Xây dựng model

In [45]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

NUM_CLASSES = 2

# load file config
config = RobertaConfig.from_pretrained(
    f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/config.json',
    from_tf=False,
    num_labels=NUM_CLASSES,
    output_hidden_states=False
)

# load model
BERT_SA = RobertaForSequenceClassification.from_pretrained(
    f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/model.bin',
    config=config
)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/pRoBERTa/PhoBERT_base_transformers/model.bin and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
BERT_SA


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=0)
      (position_embeddings): Embedding(258, 768, padding_idx=0)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [47]:
import torch
import torch.nn as nn
from transformers import RobertaModel

class CustomRobertaClassificationHead(nn.Module):
    def __init__(self, hidden_size, num_labels):
        super(CustomRobertaClassificationHead, self).__init__()
        self.dense = nn.Linear(768, 768)
        self.dropout = nn.Dropout(p=0.2)
        self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=256, batch_first=True)
        self.fc1 = nn.Linear(256, 128)
        self.out_proj = nn.Linear(128, num_labels)

    def forward(self, hidden_states):
        x = self.dense(hidden_states)  # Ánh xạ hidden_states qua lớp tuyến tính dense
        x = torch.relu(x)  # Áp dụng hàm kích hoạt ReLU
        x = self.dropout(x)  # Áp dụng dropout
        _, (h_n, _) = self.lstm(x)  # Đưa x qua LSTM và lấy hidden state cuối cùng
        x = self.fc1(h_n[-1])
        x = self.out_proj(x)

        return x

class CustomRobertaForSequenceClassification(nn.Module):
    def __init__(self, model, num_labels):
        super(CustomRobertaForSequenceClassification, self).__init__()

        self.roberta = model
        self.roberta.classifier = CustomRobertaClassificationHead(hidden_size=768, num_labels=num_labels)

    def forward(self, input_ids, token_type_ids, attention_mask, labels):
        outputs = self.roberta(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=labels)
        return outputs

    def save_pretrained(self, PATH):
       # Create the directory if it doesn't exist
      os.makedirs(PATH, exist_ok=True)

      # Save the model's state_dict to a file
      model_path = os.path.join(PATH, 'model.bin')
      torch.save(self.state_dict(), model_path)

In [48]:
custom_model = CustomRobertaForSequenceClassification(BERT_SA, 2)

# Train model

In [49]:
# Hàm đánh giá độ chính xác
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  F1_score = f1_score(pred_flat, labels_flat, average='macro')

  return accuracy_score(pred_flat, labels_flat), F1_score

In [50]:
custom_model.cuda()
print("done!")

done!


In [52]:
# định nghĩa tham số
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 1
save_dir = '/content/drive/MyDrive/pRoBERTa/trained_module/label_selection/2classes'
best_acc = 0

# lưu lại lịch sử
losses = []
train_accs = []
val_accs = []

# define optimizer
param_optimizer = list(custom_model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)

# Train theo từng epoch
for epoch_i in range(0, epochs):
  print('================= Epoch {} / {} ====================='.format(epoch_i + 1, epochs))
  print('Training ... ')

  # Tham số train
  total_loss = 0
  custom_model.train()
  train_accuracy = 0
  nb_train_steps = 0
  train_f1 = 0

  # Train từng batch
  for step, batch in enumerate(tqdm(train_dataloader)):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    custom_model.zero_grad()
    outputs = custom_model(
        b_input_ids,
        token_type_ids=None,
        attention_mask=b_input_mask,
        labels=b_labels
    )
    loss = outputs[0]
    total_loss += loss.item()

    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_train_accuracy, tmp_train_f1 = flat_accuracy(logits, label_ids)
    train_accuracy += tmp_train_accuracy
    train_f1 += tmp_train_f1
    nb_train_steps += 1

    loss.backward()
    torch.nn.utils.clip_grad_norm_(custom_model.parameters(), 1.0)
    optimizer.step()

  # tính average loss và accuracy cho epoch đó
  avg_train_loss = total_loss / len(train_dataloader)
  losses.append(avg_train_loss)
  train_accs.append(train_accuracy/nb_train_steps)

  print('Accuracy: {0:.4f}'.format(train_accuracy/nb_train_steps))
  print(" F1 score: {0:.4f}".format(train_f1/nb_train_steps))
  print(" Average training loss: {0:.4f}".format(avg_train_loss))

  print("Running Validation...")

  custom_model.eval()

  # tham số validate
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  eval_f1 = 0

  # # đánh giá
  # for batch in tqdm(val_dataloader):
  #   batch = tuple(t.to(device) for t in batch)

  #   b_input_ids, b_input_mask, b_labels = batch

  #   with torch.no_grad():
  #     outputs = custom_model(
  #         b_input_ids,
  #         token_type_ids=None,
  #         attention_mask=b_input_mask,
  #         labels=labels
  #     )
  #     logits = outputs[0]
  #     logits = logits.detach().cpu().numpy()
  #     label_ids = b_labels.to('cpu').numpy()

  #     tmp_eval_accuracy, tmp_eval_f1 = flat_accuracy(logits, label_ids)

  #     eval_accuracy += tmp_eval_accuracy
  #     eval_f1 += tmp_eval_f1
  #     nb_eval_steps += 1

  # đánh giá
  for step, batch in enumerate(tqdm(val_dataloader)):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    custom_model.zero_grad()
    outputs = custom_model(
        b_input_ids,
        token_type_ids=None,
        attention_mask=b_input_mask,
        labels=b_labels
    )
    loss = outputs[0]
    total_loss += loss.item()

    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy, tmp_eval_f1 = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    eval_f1 += tmp_eval_f1
    nb_eval_steps += 1


  avg_val_acc = eval_accuracy/nb_eval_steps
  val_accs.append(avg_val_acc)
  print(" Accuracy: {0:.4f}".format(avg_val_acc))
  print(" F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))

  # save checkpoint
  if avg_val_acc > best_acc:
    # save checkpoint
    print(f'Saved model_{avg_val_acc}_to %s' %save_dir)
    custom_model.save_pretrained(save_dir)
    best_acc = avg_val_acc

  # early stopping
  # if epoch_i >= 4: # Khởi động 3 epochs
  #   if avg_val_acc < val_accs[-2]:
  #     # bị overfit
  #     print('Early stopping!')
  #     break
print("Training complete!")

================= Epoch 1 / 1 =====================
Training ... 


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/3164 [00:00<?, ?it/s]

Accuracy: 0.8880
 F1 score: 0.7791
 Average training loss: 0.3579
Running Validation...


  0%|          | 0/352 [00:00<?, ?it/s]

 Accuracy: 0.8970
 F1 score: 0.7900
Saved model_0.8970170454545454_to /content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection/2classes
Training complete!
